In [ ]:
import geopandas as gpd
import os
import logging
import shapely.wkt as wkt

from utils.geometry_utils import create_square_polygon, create_bounding_box_from_polygon
from utils.filename_utils import clean_filename

from src.download_maps_images import GoogleMaps, MapboxRasterTiles, MapboxStaticTiles, GeoTif

from config.config import (
    DEFAULT_ZOOM, 
    DEFAULT_SQUARE_SIZE_KM, 
    DEFAULT_WIDTH_KM, 
    DEFAULT_HEIGHT_KM, 
    DEFAULT_OUTPUT_DIR, 
    shapefile_path, 
    output_dir, 
    index_to_use, 
    gdf
)

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')



### Google Maps API

In [1]:
os.makedirs(output_dir, exist_ok=True)
gdf1 = gdf[:3]

for index, row in gdf1.iterrows():
    square_wkt = create_square_polygon(str(row['geometry']), square_size_km=DEFAULT_SQUARE_SIZE_KM)
    parcel_name = row["parcel_id"]
    
    GoogleMaps.get_static_map(output_dir, parcel_name, index, square_wkt)

logging.info("Download complete")


2024-05-22 02:08:50,320 - INFO - Tile downloaded successfully: staticapi_zoom16/0_62f6d924-a257-4847-bb67-ad63c1eebb21.png
2024-05-22 02:08:50,982 - INFO - Tile downloaded successfully: staticapi_zoom16/1_fa2d3722-d8d3-4787-9ac1-69aaadd8962b.png
2024-05-22 02:08:51,745 - INFO - Tile downloaded successfully: staticapi_zoom16/2_3f306e5e-6828-4faa-9e23-07ce03ad4b9a.png
2024-05-22 02:08:51,746 - INFO - Download complete


### MapBox Raster API

In [ ]:
original_polygon = gdf.iloc[index_to_use]["geometry"]
square_polygon_wkt = create_square_polygon(str(original_polygon), square_size_km=1.5)
polygon_obj = wkt.loads(square_polygon_wkt)
latitude, longitude = polygon_obj.centroid.y, polygon_obj.centroid.x
x, y = MapboxRasterTiles.lat_lon_to_tile_xy(latitude, longitude, DEFAULT_ZOOM)
output_dir = "/Users/Hiba/Desktop/tourba/mapbox"
os.makedirs(output_dir, exist_ok=True)
MapboxRasterTiles.download_raster_tile(output_dir, gdf.iloc[index_to_use]["parcel_id"], index_to_use, x, y)

logging.info("Download complete")




2024-05-22 02:18:27,832 - INFO - Tile downloaded successfully: /Users/Hiba/Desktop/tourba/mapbox/145_e4a605f3-c01d-4368-a822-e712172cc94d.png
2024-05-22 02:18:27,833 - INFO - Download complete


### MapBox Static API

In [ ]:
for index, row in gdf[:1].iterrows():
    square_polygon_wkt = create_square_polygon(str(row['geometry']), square_size_km=2)
    parcel_name = row['parcel_id']
    MapboxStaticTiles.get_static_map(output_dir, parcel_name, index, square_polygon_wkt)
    
logging.info("Download complete")


2024-05-22 02:24:40,035 - ERROR - Error while downloading: 404. URL : https://maps.googleapis.com/maps/api/staticmap/-4.72846163227669,33.990386880017766,16/640x640?access_token=pk.eyJ1IjoiaGliYWxiIiwiYSI6ImNsdm9nMXVtYTBqODUyam8xNWVucWNtZXUifQ.j2dv4_4_rcZFyINA4wzJ8g
2024-05-22 02:24:40,036 - INFO - Download complete


### SAM Geo

In [ ]:
os.makedirs(DEFAULT_OUTPUT_DIR, exist_ok=True)
gdf1 = gdf[:3]
for index, row in gdf1.iterrows():
    bbox = create_bounding_box_from_polygon(str(row['geometry']), DEFAULT_WIDTH_KM, DEFAULT_HEIGHT_KM)
    file_name = clean_filename(row["parcel_id"])
    output_path = f"{DEFAULT_OUTPUT_DIR}/{index}_{file_name}.png"
    GeoTif.geotif(output=output_path, bbox=bbox, zoom=DEFAULT_ZOOM)

logging.info("Download complete")
 

2024-05-22 02:32:06,360 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31905&y=26183&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:06,365 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31905&y=26181&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:06,366 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31905&y=26182&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:06,368 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31905&y=26180&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:06,373 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31906&y=26180&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:06,411 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31906&y=26181&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:06,432 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31906&y=26182&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:06,435 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31906&y=26183&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32

Downloaded image 01/16
Downloaded image 02/16
Downloaded image 03/16
Downloaded image 04/16
Downloaded image 05/16
Downloaded image 06/16
Downloaded image 07/16
Downloaded image 08/16
Downloaded image 09/16
Downloaded image 10/16
Downloaded image 11/16
Downloaded image 12/16
Downloaded image 13/16
Downloaded image 14/16
Downloaded image 15/16
Downloaded image 16/16
Saving GeoTIFF. Please wait...


2024-05-22 02:32:07,536 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31907&y=26181&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:07,542 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31907&y=26179&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:07,543 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31907&y=26178&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:07,543 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31908&y=26178&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:07,562 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31907&y=26180&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:07,568 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31908&y=26179&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:07,577 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31908&y=26180&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:07,581 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31908&y=26181&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32

Image saved to ZOOM17/0_62f6d924-a257-4847-bb67-ad63c1eebb21.png
Downloaded image 01/16
Downloaded image 02/16
Downloaded image 03/16
Downloaded image 04/16
Downloaded image 05/16
Downloaded image 06/16
Downloaded image 07/16
Downloaded image 08/16
Downloaded image 09/16


2024-05-22 02:32:07,610 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31909&y=26180&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:07,621 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31909&y=26181&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:07,624 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31910&y=26178&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:07,630 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31910&y=26179&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:07,655 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31910&y=26181&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:07,663 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31910&y=26180&z=16 "HTTP/1.1 200 OK"


...

Image saved to ZOOM17/1_fa2d3722-d8d3-4787-9ac1-69aaadd8962b.png
Downloaded image 01/16
Downloaded image 02/16
Downloaded image 03/16
Downloaded image 04/16
Downloaded image 05/16
Downloaded image 06/16


2024-05-22 02:32:08,876 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31920&y=26194&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:08,880 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31921&y=26191&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:08,892 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31921&y=26192&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:08,900 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31921&y=26193&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:08,940 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31922&y=26191&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:08,968 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31921&y=26194&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:08,970 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31922&y=26193&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32:08,977 - INFO - HTTP Request: GET https://mt1.google.com/vt/lyrs=s&x=31922&y=26192&z=16 "HTTP/1.1 200 OK"
2024-05-22 02:32

Downloaded image 07/16
Downloaded image 08/16
Downloaded image 09/16
Downloaded image 10/16
Downloaded image 11/16
Downloaded image 12/16
Downloaded image 13/16
Downloaded image 14/16
Downloaded image 15/16
Downloaded image 16/16
Saving GeoTIFF. Please wait...


2024-05-22 02:32:09,445 - INFO - Download complete


Image saved to ZOOM17/2_3f306e5e-6828-4faa-9e23-07ce03ad4b9a.png
